In [1]:
! pip install kaggle

In [ ]:
from google.colab import files

# Kaggle API anahtarını yükleyin
files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shadyeldakrory/shadyfinal3

In [5]:
import zipfile
import os


zip_path = '/content/shadyfinal3.zip'

# Çıkartılacak dizini belirtin
extracted_folder_path = '/content/shadyfinal3'

# Zip dosyasını çıkart
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Çıkartılan dosyaları göster
os.listdir(extracted_folder_path)

['test', 'validation', 'train']

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D,RandomFlip,RandomRotation
from keras.applications import MobileNet,resnet,MobileNetV2,DenseNet121,EfficientNetB3
from tensorflow.keras.applications import ResNet50
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/shadyfinal3/train',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/shadyfinal3/validation',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical')

In [8]:
data_augmentation =Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.6),

])
model = Sequential([
  # Add the preprocessing layers you created earlier.
  #data_augmentation,
    #DenseNet121(weights='imagenet',include_top=False),
    #ResNet50(weights='imagenet',include_top=False),
    MobileNetV2(weights='imagenet',include_top=False),
    GlobalAveragePooling2D(),
    Dense(1024,activation='relu'),
    Dense(1024,activation='relu'),
    Dropout(0.2),
    Dense(512,activation='relu'),
    Dense(512,activation='relu'),
    Dropout(0.2),
    Dense(256,activation='relu'),
    Dense(4,activation='softmax')

  # Rest of your model.
])

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    y_true = K.argmax(y_true, axis=1)
    y_pred = K.argmax(y_pred, axis=1)

    true_positives = K.sum(K.cast(K.equal(y_true, 1) & K.equal(y_pred, 1), dtype=tf.float32))
    possible_positives = K.sum(K.cast(K.equal(y_true, 1), dtype=tf.float32))
    predicted_positives = K.sum(K.cast(K.equal(y_pred, 1), dtype=tf.float32))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    return f1

In [10]:
validation_steps = len(validation_generator)
steps_per_epoch = len(train_generator)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), f1_metric])
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps)

<ipython-input-10-8a5114299081>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
237/237 [==============================] - 305s 1s/step - loss: 0.6221 - accuracy: 0.7538 - precision: 0.8850 - recall: 0.5955 - f1_metric: 0.7025 - val_loss: 1.3371 - val_accuracy: 0.5646 - val_precision: 0.5795 - val_recall: 0.5423 - val_f1_metric: 0.5851
Epoch 2/5
237/237 [==============================] - 272s 1s/step - loss: 0.2145 - accuracy: 0.9211 - precision: 0.9301 - recall: 0.9112 - f1_metric: 0.8809 - val_loss: 1.3451 - val_accuracy: 0.6072 - val_precision: 0.6114 - val_recall: 0.6030 - val_f1_metric: 0.6353
Epoch 3/5
237/237 [==============================] - 276s 1s/step - loss: 0.1408 - accuracy: 0.9494 - precision: 0.9567 - recall: 0.9445 - f1_metric: 0.9270 - val_loss: 1.2882 - val_accuracy: 0.6302 - val_precision: 0.6352 - val_recall: 0.6252 - val_f1_metric: 0.6635
Epoch 4/5
237/237 [==============================] - 270s 1s/step - loss: 0.1041 - accuracy: 0.9626 - precision: 0.9678 - recall: 0.9580 - f1_metric: 0.9446 - val_loss: 0.6785 - val_accuracy: 0.77